In [1]:
# Import libraries
import os
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

from tqdm import tqdm

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", tf.test.gpu_device_name())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU memory growth enabled.")
    except RuntimeError as e:
        print(e)

2025-03-30 13:01:51.573172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743364911.658791     613 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743364911.687827     613 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 13:01:51.892215: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available: 1
TensorFlow is using: /device:GPU:0
GPU memory growth enabled.


I0000 00:00:1743364916.055682     613 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1743364916.057844     613 gpu_device.cc:2022] Created device /device:GPU:0 with 5564 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:06:00.0, compute capability: 8.6


In [2]:
def load_data(spectrogram_dir, label_dir):
    spectrograms = []
    labels = []

    # Load spectrograms
    spectrogram_files = sorted(os.listdir(spectrogram_dir))
    label_files = sorted(os.listdir(label_dir))

    total_files = len(spectrogram_files)

    for spec_file, label_file in tqdm(zip(spectrogram_files, label_files), 
                                        total=total_files, 
                                        desc="Loading Data", 
                                        unit="file"):
        # Load data
        spec_data = np.load(os.path.join(spectrogram_dir, spec_file))
        label_data = np.load(os.path.join(label_dir, label_file)).T

        # Append if there is valid data
        if len(spec_data) > 0:
            spectrograms.append(spec_data)
            labels.append(label_data)

    # Convert to numpy arrays 
    if spectrograms:
        X = np.vstack(spectrograms)
        Y = np.vstack(labels)
        return X, Y
    else:
        return np.array([]), np.array([])

In [3]:
X_train, Y_train = load_data("spectrograms_train", "labels_train")
X_train = np.expand_dims(X_train, axis=-1)
print("Final X_train:", X_train.shape, "Final Y_train:", Y_train.shape)

Loading Data: 100%|██████████| 1289/1289 [00:04<00:00, 272.61file/s]


Final X_train: (319508, 64, 42, 1) Final Y_train: (319508, 8)


In [4]:
# Occurrences
class_counts = np.sum(Y_train, axis=0)
total_samples = len(Y_train)

for i, count in enumerate(class_counts):
    print(f"Class {i}: {count} occurrences ({count / total_samples:.2%} of the data)")

Class 0: 128695 occurrences (40.28% of the data)
Class 1: 13526 occurrences (4.23% of the data)
Class 2: 27686 occurrences (8.67% of the data)
Class 3: 125932 occurrences (39.41% of the data)
Class 4: 171444 occurrences (53.66% of the data)
Class 5: 257671 occurrences (80.65% of the data)
Class 6: 155212 occurrences (48.58% of the data)
Class 7: 271620 occurrences (85.01% of the data)


In [11]:
model = Sequential([
    Input(shape=(64, 42, 1)),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.0001)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.0001)),
    Dropout(0.4),  
    Dense(8, activation='sigmoid', dtype='float32')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 64, 42, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 32, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 16, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 8, 5, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 5120)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       655,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 749,192 (2.86 MB)

 Trainable params: 749,192 (2.86 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train model
history = model.fit(X_train, Y_train, batch_size=64, epochs=10, verbose=1)

# Final loss & accuracy
final_loss = history.history['loss'][-1]
final_accuracy = history.history['binary_accuracy'][-1]
print(f"Final Loss: {final_loss:.4f}, Final Accuracy: {final_accuracy:.4f}")

2025-03-30 13:02:07.912350: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3435350016 exceeds 10% of free system memory.
2025-03-30 13:02:09.656315: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3435350016 exceeds 10% of free system memory.


Epoch 1/10


I0000 00:00:1743364931.481981     886 service.cc:148] XLA service 0x7fdd40006070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743364931.482991     886 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2025-03-30 13:02:11.535679: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743364931.722641     886 cuda_dnn.cc:529] Loaded cuDNN version 90300


  33/4993 ━━━━━━━━━━━━━━━━━━━━ 23s 5ms/step - binary_accuracy: 0.6750 - loss: 0.9377

I0000 00:00:1743364934.097616     886 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4993/4993 ━━━━━━━━━━━━━━━━━━━━ 31s 5ms/step - binary_accuracy: 0.8362 - loss: 0.3848
Epoch 2/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - binary_accuracy: 0.8907 - loss: 0.2849
Epoch 3/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - binary_accuracy: 0.8997 - loss: 0.2709
Epoch 4/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - binary_accuracy: 0.9041 - loss: 0.2645
Epoch 5/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - binary_accuracy: 0.9068 - loss: 0.2600
Epoch 6/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - binary_accuracy: 0.9081 - loss: 0.2580
Epoch 7/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - binary_accuracy: 0.9090 - loss: 0.2565
Epoch 8/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - binary_accuracy: 0.9102 - loss: 0.2544
Epoch 9/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - binary_accuracy: 0.9109 - loss: 0.2532
Epoch 10/10
4993/4993 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - binary_accuracy: 0.9113 - loss: 0.2527
Final Loss: 0.2534, Final Accuracy: 0.9109


In [7]:
# Get predictions
Y_pred = model.predict(X_train)

# Convert to binary (threshold the probabilities)
threshold = 0.5
Y_pred_binary = (Y_pred > threshold).astype(int)

# Calculate metrics
precision = precision_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
recall = recall_score(Y_train, Y_pred_binary, average='samples', zero_division=0)
f1 = f1_score(Y_train, Y_pred_binary, average='samples', zero_division=0)

# Print the metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

2025-03-30 13:06:33.577388: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3435350016 exceeds 10% of free system memory.
2025-03-30 13:06:35.108105: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3435350016 exceeds 10% of free system memory.


9985/9985 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step
Precision: 0.8930
Recall: 0.8677
F1-Score: 0.8699


In [8]:
X_test, Y_test = load_data("spectrograms_test", "labels_test")
X_test = np.expand_dims(X_test, axis=-1)
Y_pred_test = model.predict(X_test)

Loading Data: 100%|██████████| 151/151 [00:00<00:00, 274.88file/s]


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [9]:
# --- Threshold Adjustment ---
num_classes = Y_test.shape[1]
thresholds = np.array([0.5] * num_classes)  # Default 0.5
# Adjust thresholds as needed (example):
thresholds[0] = 0.5
thresholds[1] = 0.3
thresholds[2] = 0.3
thresholds[3] = 0.3
thresholds[4] = 0.6
thresholds[6] = 0.4

Y_pred_test_binary = (Y_pred_test >= thresholds).astype(int)
# --- End Threshold Adjustment ---

precision_test = precision_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
recall_test = recall_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)
f1_test = f1_score(Y_test, Y_pred_test_binary, average='samples', zero_division=0)

print(f"Test Precision: {precision_test:.4f}")
print(f"Test Recall: {recall_test:.4f}")
print(f"Test F1-Score: {f1_test:.4f}")

# Compute per-class precision, recall, and F1-score
precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
    Y_test, Y_pred_test_binary, average=None, zero_division=0
)

# Occurrences
class_counts = np.sum(Y_train, axis=0) # Make sure Y_train is defined.
total_samples = len(Y_train)

for i in range(num_classes):
    print(f"Class {i} | Precision={precision_per_class[i]:.4f}, Recall={recall_per_class[i]:.4f}, "
          f"F1={f1_per_class[i]:.4f} | Thresh={thresholds[i]:.1f}, Occurrences={int(class_counts[i])} ({class_counts[i] / total_samples:.2%})")

Test Precision: 0.8768
Test Recall: 0.8462
Test F1-Score: 0.8481
Class 0 | Precision=0.7812, Recall=0.7650, F1=0.7730 | Thresh=0.5, Occurrences=128695 (40.28%)
Class 1 | Precision=0.8212, Recall=0.7936, F1=0.8072 | Thresh=0.3, Occurrences=13526 (4.23%)
Class 2 | Precision=0.8349, Recall=0.6932, F1=0.7575 | Thresh=0.3, Occurrences=27686 (8.67%)
Class 3 | Precision=0.8144, Recall=0.8266, F1=0.8204 | Thresh=0.3, Occurrences=125932 (39.41%)
Class 4 | Precision=0.8485, Recall=0.8019, F1=0.8245 | Thresh=0.6, Occurrences=171444 (53.66%)
Class 5 | Precision=0.9537, Recall=0.9764, F1=0.9649 | Thresh=0.5, Occurrences=257671 (80.65%)
Class 6 | Precision=0.9763, Recall=0.7589, F1=0.8540 | Thresh=0.4, Occurrences=155212 (48.58%)
Class 7 | Precision=0.9742, Recall=0.9725, F1=0.9734 | Thresh=0.5, Occurrences=271620 (85.01%)


In [19]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 64, 42, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 21, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 21, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 10, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 10, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       163,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 563,162 (2.15 MB)

 Trainable params: 187,720 (733.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 375,442 (1.43 MB)

In [20]:
model.save("instrument_classifier.h5")
model = tf.keras.models.load_model("instrument_classifier.h5", compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("instrument_classifier.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxmix4c38/assets


INFO:tensorflow:Assets written to: /tmp/tmpxmix4c38/assets


Saved artifact at '/tmp/tmpxmix4c38'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 42, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  139987550737104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550742560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139986066711376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139986066716480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550784144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550784848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550795056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550797168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550873280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139987550871344: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743331319.240398    2088 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743331319.240491    2088 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-30 03:41:59.242180: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxmix4c38
2025-03-30 03:41:59.242940: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-30 03:41:59.242951: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpxmix4c38
I0000 00:00:1743331319.248159    2088 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-30 03:41:59.249256: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-30 03:41:59.278077: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpxmix4c38
2025-03-30 03:41:59.286863: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 